# Audio Recorder and Analyzer

In [1]:
import pyaudio
import wave
import numpy as np
import matplotlib.pyplot as plt
from scipy import fftpack 
import time
from tkinter import TclError
%matplotlib qt

In [2]:
# input devices
import sounddevice as sd
print(sd.query_devices()) 

# from the output list select the number of the desired input device and give it to the variable "INDEX"

   0 Microsoft Sound Mapper - Input, MME (2 in, 0 out)
>  1 Linea in ingresso (Realtek(R) A, MME (2 in, 0 out)
   2 Linea (NewTek NDI Audio), MME (2 in, 0 out)
   3 Microfono (Logitech G433 Gaming, MME (2 in, 0 out)
   4 Microfono (NVIDIA Broadcast), MME (2 in, 0 out)
   5 Microfono (DroidCam Virtual Aud, MME (1 in, 0 out)
   6 Microsoft Sound Mapper - Output, MME (0 in, 2 out)
<  7 Realtek HD Audio 2nd output (Re, MME (0 in, 2 out)
   8 Altoparlanti (Logitech G433 Gam, MME (0 in, 8 out)
   9 Speakers (NVIDIA Broadcast), MME (0 in, 2 out)
  10 ITMF27V14QHD (NVIDIA High Defin, MME (0 in, 2 out)
  11 Realtek Digital Output (Realtek, MME (0 in, 2 out)
  12 Output (RTX-Audio Point), Windows WDM-KS (0 in, 2 out)
  13 Microphone (RTX-Audio Point), Windows WDM-KS (2 in, 0 out)
  14 Output (NVIDIA High Definition Audio), Windows WDM-KS (0 in, 2 out)
  15 Microfono (), Windows WDM-KS (2 in, 0 out)
  16 Altoparlanti (), Windows WDM-KS (0 in, 8 out)
  17 Linea in ingresso (Realtek HD Audio Line i

## Audio recording

In [3]:
# constants
INDEX = 1                           # audio device index
CHUNK = 1024 * 2                    # how many audio samples per frame we display 
FORMAT = pyaudio.paInt16            # 16bit format per sample
CHANNELS = 1                        # single channel for microphone
RATE = 44100                        # samples per second [Hz] (common choice)
rec_time = 10                       # recording time [seconds]
delay_time = 2                      # time after which it starts recording [seconds]
clipping_start = 0                  # approximate value at which clipping begins (to be found from the graph) 

# output file
filename = 'output.wav'

In [11]:
# pyaudio class instance
p = pyaudio.PyAudio()

# stream object to get data from microphone
stream = p.open(
    format=FORMAT,
    channels=CHANNELS,
    rate=RATE,
    input=True,
    frames_per_buffer=CHUNK,
    input_device_index=INDEX
)

# COLLECT AUDIO DATA
start_time = time.time()

# delay
print('Process started')
while True:
    current_time = time.time()
    elapsed_delaytime = current_time - start_time
    if elapsed_delaytime > delay_time: 
        break
    
# recording
print('Start recording (after t=%is)' %elapsed_delaytime)

frames = []
while True:
    current_time = time.time()
    elapsed_time = current_time - (start_time + delay_time)
    
    if elapsed_time > rec_time: 
        break
    
    # binary data
    data = stream.read(CHUNK)   # read 1 chunk at a time
    frames.append(data)
    
print('Finished recording')
print('Recorded for %i seconds' %elapsed_time)
stream.stop_stream()
stream.close()
p.terminate()  

# Save the recorded data as a WAV file
wf = wave.open(filename, 'wb')
wf.setnchannels(CHANNELS)
wf.setsampwidth(p.get_sample_size(FORMAT))
wf.setframerate(RATE)
wf.writeframes(b''.join(frames))
wf.close()

# join the frames and convert to integers
amplitude = np.frombuffer(b''.join(frames), dtype=np.int16)

Process started
Start recording (after t=2s)
Finished recording
Recorded for 10 seconds


## Waveform and spectrogram plots

In [12]:
# plot the signal
fig, ax0 = plt.subplots(1, figsize=(15, 8))
ax0.set_title('AUDIO WAVEFORM', fontsize=15)
ax0.set_xlabel('Samples', fontsize=14)
ax0.set_ylabel('Amplitude', fontsize=14)
ax0.plot(amplitude)
plt.show()
plt.savefig('./output_images/waveform.png',  bbox_inches='tight')

# plot the spectrogram
fig, ax1 = plt.subplots(1, figsize=(15, 8))
ax1.set_title('SPECTROGRAM', fontsize=15)
ax1.specgram(amplitude)
plt.show()
plt.savefig('./output_images/spectrogram.png', bbox_inches='tight')
    

Note that the x-axis of the waveform is just the np_array indices. If we wanted to see the time in seconds instead, we could divide each x-axis value by the sample rate.

## FFT of the signal

In [13]:
# FFT of the signal
sig_fft = fftpack.fft(amplitude[clipping_start:])

# power (sig_fft is of complex dtype)
power = np.abs(sig_fft) # these are the "weights" of the Fourier components for each discrete frequency interval

# return the corresponding frequencies
sample_freq = fftpack.fftfreq(amplitude[clipping_start:].size)

# plot the FFT power
plt.figure(figsize=(15, 8))
plt.plot(sample_freq, power)
plt.title('FFT of the signal', fontsize=15)
plt.xlabel('Frequency [Hz]', fontsize=14)
plt.ylabel('Power', fontsize=14)

# find the peak frequency: focus only on only the positive frequencies
pos_mask = np.where(sample_freq > 0)
freqs = sample_freq[pos_mask]
pos_power = power[pos_mask]
# find the max of freqs array
peak_freq = freqs[pos_power.argmax()]
print("Peak frequency: %.6f" %peak_freq)

# inner plot to show the peak frequency
axes = plt.axes([0.62, 0.42, 0.25, 0.4])
plt.title('Peak frequency', fontsize=14)
plt.plot(freqs[:5000], pos_power[:5000])
plt.setp(axes, yticks=[])
plt.show()
plt.savefig('output_images/FFT.png', bbox_inches='tight')

# scipy.signal.find_peaks_cwt can also be used for more advanced peak detection

Peak frequency: 0.009070


## Inverse FFT 

In [ ]:
high_freq_fft = sig_fft.copy()
# filter (set to zero) all high frequency components that are larger than peak_freq
high_freq_fft[np.abs(sample_freq) > peak_freq] = 0
# calculate the Inverse Fast Fourier Transform
filtered_sig = fftpack.ifft(high_freq_fft)
# only take the real part
real_filtered_signal = np.real(filtered_sig)

# plot the result of the IFFT
plot_range_min = 200000
plot_range_max = 200000+1000
plt.figure(figsize=(15, 8))
plt.plot(amplitude[plot_range_min:plot_range_max], alpha=0.5, label='Original signal')
plt.plot(real_filtered_signal[plot_range_min:plot_range_max], linewidth=3, label='Filtered signal (FTT)')
plt.xlabel('Samples')
plt.ylabel('Amplitude')
plt.legend(loc='best')
plt.show()